In [1]:
import matplotlib.pyplot as plt
import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score, silhouette_score
import optuna
from joblib import Parallel, delayed
from utils.Loader import NEUDataset
from utils.Perspectiver import Perspectiver
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import multiprocessing
import torch

/home/lingfeng/Desktop/pytorch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = NEUDataset(set="train", transform=None, seed=1)

# Probar con un DataLoader
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Intentar iterar sobre el dataloader
for images, labels in dataloader:
    print("Iteración exitosa.")
    print("Imagen shape:", images.shape)
    print("Etiqueta shape:", labels.shape)
    break 

Iteración exitosa.
Imagen shape: torch.Size([4, 1, 200, 200])
Etiqueta shape: torch.Size([4, 6])


In [3]:
image, label = dataset.__getitem__(index=2)
original_image = Perspectiver.grayscale_to_rgb(Perspectiver.normalize_to_uint8(image.detach().cpu().numpy()[0]))

In [4]:
def optimize_image(image, n_tries: int = 100):
    """
    Optimizes the mean shift parameters to maximize the metric: (Silhouette Score / Number of Clusters).

    Args:
        image (np.array): The input image with shape (200, 200, 3).
        n_tries (int): The number of optimization trials to run.

    Returns:
        dict: The best parameters and the best score.
    """

    # Define the objective function for Optuna
    def objective(trial):
        # Suggest parameters for spatial radius and color radius
        sp = trial.suggest_float("sp", 5.0, 50.0)  # Spatial window radius
        sr = trial.suggest_float("sr", 5.0, 50.0)  # Color window radius

        # Apply mean shift filtering
        after = Perspectiver.meanShift(image, sp, sr)

        # Evaluate clustering
        scores = Perspectiver.evaluate_clustering(image, after)
        silhouette_score = scores["Calinski-Harabasz Index"]
        n_clusters = len(np.unique(after))

        # Avoid division by zero (in case of degenerate clustering)
        if n_clusters == 0:
            return float("-inf")

        # Metric to maximize: Silhouette Score per cluster
        metric_to_maximize = silhouette_score / n_clusters
        return metric_to_maximize

    # Determine the number of CPU threads
    n_jobs = multiprocessing.cpu_count()
    print(f"Using {n_jobs} CPU threads for parallel optimization.")

    # Run the optimization with parallel trials
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_tries, n_jobs=n_jobs)

    # Return the best parameters and the best score
    return {
        "best_params": study.best_params,
        "best_score": study.best_value
    }


In [5]:
optimize_image(original_image)


[I 2025-01-28 14:05:38,321] A new study created in memory with name: no-name-0c907e8b-eb53-4b31-88f4-e3b856fb38db


Using 8 CPU threads for parallel optimization.


[I 2025-01-28 14:07:06,121] Trial 1 finished with value: 5.7683091001021625 and parameters: {'sp': 5.1621705859518014, 'sr': 38.68110670298585}. Best is trial 1 with value: 5.7683091001021625.
[I 2025-01-28 14:07:06,415] Trial 5 finished with value: 9.996372396766976 and parameters: {'sp': 47.86607782388755, 'sr': 49.91535596233613}. Best is trial 5 with value: 9.996372396766976.
[I 2025-01-28 14:07:06,955] Trial 4 finished with value: 6.883759389565641 and parameters: {'sp': 24.189374253945452, 'sr': 33.60874269103133}. Best is trial 5 with value: 9.996372396766976.
[I 2025-01-28 14:07:07,771] Trial 7 finished with value: 20.63502800793958 and parameters: {'sp': 13.832401608720112, 'sr': 17.402148929492114}. Best is trial 7 with value: 20.63502800793958.
[I 2025-01-28 14:07:08,096] Trial 0 finished with value: 66.37493160422407 and parameters: {'sp': 18.332693949169276, 'sr': 12.545853913960176}. Best is trial 0 with value: 66.37493160422407.
[I 2025-01-28 14:07:08,267] Trial 3 finish

{'best_params': {'sp': 44.932586235494384, 'sr': 5.010407483180379},
 'best_score': 205.4640538292614}

In [ ]:
Perspectiver.plotComparison(original_image, Perspectiver.dbscanClustering(Perspectiver.meanShift(original_image,44.932586235494384, 5.010407483180379), 100, 100))